In [1]:
import pandas as pd



from constants import excel_path
# from constants import sheet_name
# from constants import okr_sheet_name

from utils import get_df_roadshow

In [2]:
# Get the Excel file
xls = pd.ExcelFile(excel_path)

# Get sheet names
sheet_names = xls.sheet_names
print(sheet_names)  # e.g., ['Sheet1', 'Sheet2', ...]

# Close the Excel file
xls.close()

['研究员个人维度工作量统计', '行业维度工作量统计', '销售服务量统计', '专家使用量统计', '路演', '晨会接入', '客户内外部策略会', '调研', '课题', '54A电话服务', '其他服务', '报告', '电话会议', '电话服务明细', '晨会发言', '晨会缺勤', '2023年1月专家费']


In [3]:
# read shodshow sheet
# read 'xuhao' as int
df = pd.read_excel(excel_path, sheet_name='路演')
df.head()

,序号,日期,开始时间,结束时间,服务事项,研究员,研究员.1,研究员.2,研究员.3,研究员.4,...,所属团队,客户机构,客户分级,客户区域,对口销售,参与人员,主题,备注,创建人,Unnamed: 21
0,1.0,2023-01-11,15:00:00,16:00:00,路演-线上路演,张航,NaN,NaN,NaN,NaN,...,金属,安联保险资产管理有限公司,A,北京,张航旗,1,1,NaN,张航旗,2023-01-10 10:34:51
1,2.0,2023-01-17,13:30:00,14:30:00,路演-线上路演,许睿,NaN,NaN,NaN,NaN,...,医药,安联保险资产管理有限公司,A,北京,张航旗,韩成盛,策略,NaN,张航旗,2023-01-15 16:02:30
2,3.0,2023-01-18,13:30:00,14:30:00,路演-线下路演,汪海洋,NaN,NaN,NaN,NaN,...,家电,安联保险资产管理有限公司,A,北京,张航旗,刘家仪,海信,NaN,张航旗,2023-01-13 15:59:11
3,4.0,2023-01-12,10:30:00,11:15:00,路演-线下路演,叶尔乐,NaN,NaN,NaN,NaN,...,金融工程,安联资管,A,北京,张航旗,汪鹏,主题,NaN,张航旗,2023-01-08 20:45:16
4,5.0,2023-01-04,11:30:00,12:30:00,路演-餐叙路演,徐皓亮,NaN,NaN,NaN,NaN,...,轻工,安信基金,3A,广深,程嘉琪,hcx,1,1,程嘉琪,2022-12-26 16:32:44


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1851 entries, 0 to 1850
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   序号           1833 non-null   float64
 1   日期           1833 non-null   object 
 2   开始时间         1833 non-null   object 
 3   结束时间         1833 non-null   object 
 4   服务事项         1851 non-null   object 
 5   研究员          1833 non-null   object 
 6   研究员.1        126 non-null    object 
 7   研究员.2        24 non-null     object 
 8   研究员.3        4 non-null      object 
 9   研究员.4        1 non-null      object 
 10  研究员.5        1 non-null      object 
 11  研究员.6        0 non-null      float64
 12  所属团队         1804 non-null   object 
 13  客户机构         1851 non-null   object 
 14  客户分级         1851 non-null   object 
 15  客户区域         1850 non-null   object 
 16  对口销售         1851 non-null   object 
 17  参与人员         1851 non-null   object 
 18  主题           1851 non-null   object 
 19  备注    

df_temp.tail()

In [5]:
df_roadshow = get_df_roadshow(df)

In [6]:
df_roadshow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   序号      1989 non-null   float64
 1   所属团队    1989 non-null   object 
 2   服务事项    1989 non-null   object 
 3   客户机构    1989 non-null   object 
 4   客户分级    1989 non-null   object 
 5   客户区域    1988 non-null   object 
 6   研究员     1989 non-null   object 
dtypes: float64(1), object(6)
memory usage: 108.9+ KB


In [7]:
df_roadshow.head()

,序号,所属团队,服务事项,客户机构,客户分级,客户区域,研究员
0,1.0,金属,路演-线上路演,安联保险资产管理有限公司,A,北京,张航
1,2.0,医药,路演-线上路演,安联保险资产管理有限公司,A,北京,许睿
2,3.0,家电,路演-线下路演,安联保险资产管理有限公司,A,北京,汪海洋
3,4.0,金融工程,路演-线下路演,安联资管,A,北京,叶尔乐
4,5.0,轻工,路演-餐叙路演,安信基金,3A,广深,徐皓亮
